# Exercise 4

Coded by:
1. Carmen Eitel - carmen.eitel@student.kit.edu

on 01.05.2019

version 1.0

## Bibliotheken und Einstellungen

In [101]:
# import libraries 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

## Teilaufgaben

### a) Daten einlesen und erkunden
Lesen Sie den Datensatz Credit.csv ein. Erkunden Sie die Daten und konvertieren Sie gegebenenfalls kategorische Variablen in ein geeignetes Format (z.B. mit pandas.get_dummies() oder sklearn.model_selection.OneHotEncoder()). 

In [29]:
# load dataset
data_raw = pd.read_csv('./data/Credit.csv')

# first insight of data by checking the first 5 data samples
data_raw.head()

,Unnamed: 0,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,3,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,4,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


Es gibt 10 unabhängige Variablen: Income (float), Limit (integer), Rating (integer), Cards, (integer), Age (integer), Education (integer), Gender (binary), Student (binary), Married (binary), Ethicity (categorical) und eine abhängige Variable: Balance (integer).

Die binären Variablen müssen in eine Dummy-Variable mit 1 und 0 bzw. True und False überführt werden. Die kategorische Variable muss in verschiedene Dummy-Variablen entsprechend der Anzahl ihrer Ausprägungen überführt werden (siehe Teilaufgabe c).

Im Folgenden werden die binären Variablen in eine Dummy-Variable überführt zusammen micht den integers und floats in ein Dataframe überführt.

In [30]:
#create dummies of gender
gender_df = pd.get_dummies(data_raw["Gender"])
gender_df.head()


,Female,Male
0,0,1
1,1,0
2,0,1
3,1,0
4,0,1


In [31]:
# select one dummy
is_male = gender_df[["Male"]]

#rename column
is_male.rename(columns = {"Male":"IsMale"}, inplace = True)
is_male.head()

,IsMale
0,1
1,0
2,1
3,0
4,1


In [32]:
# create dummies for student
student = pd.get_dummies(data_raw["Student"])
student.head()

,No,Yes
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0


In [33]:
# select one dummy
is_student = student[["Yes"]]

# rename column
is_student.rename(columns = {"Yes":"IsStudent"}, inplace = True)
is_student.head()

,IsStudent
0,0
1,1
2,0
3,0
4,0


In [34]:
# create dummies for married
married = pd.get_dummies(data_raw["Married"])
married.head()

,No,Yes
0,0,1
1,0,1
2,1,0
3,1,0
4,0,1


In [35]:
# select one dummy for married
is_married = married[["Yes"]]

# rename column
is_married.rename(columns = {"Yes":"IsMarried"}, inplace = True)
is_married.head()

,IsMarried
0,1
1,1
2,0
3,0
4,1


In [36]:
data_without_categorical_binary_variables = data_raw[["Balance","Income", "Limit", "Rating", "Cards", "Age", "Education"]]
data_without_categorical_variables = pd.concat([data_without_categorical_binary_variables, is_married, is_student, is_male], axis=1, join='inner')
data_without_categorical_variables.head()

,Balance,Income,Limit,Rating,Cards,Age,Education,IsMarried,IsStudent,IsMale
0,333,14.891,3606,283,2,34,11,1,0,1
1,903,106.025,6645,483,3,82,15,1,1,0
2,580,104.593,7075,514,4,71,11,0,0,1
3,964,148.924,9504,681,3,36,11,0,0,0
4,331,55.882,4897,357,2,68,16,1,0,1


Im Folgenden werden bereits die Dummies der kategorischen Variablen Ethnicty erstellt und ein geeignetes Dataframe erstellt. Damit wird der Datensplit in b) durchgeführt. Dies ist somit ein Vorzug aus der Teilaufgabe c).

In [37]:
# there are three values for ethnicities: African American, Asian, Caucasian
# convert them into three dummies

ethnicity_raw = pd.Series(data_raw["Ethnicity"], dtype="category") 
ethnicity_raw.head()


0    Caucasian
1        Asian
2        Asian
3        Asian
4    Caucasian
Name: Ethnicity, dtype: category
Categories (3, object): [African American, Asian, Caucasian]

In [38]:
# create dummies
african_american = pd.get_dummies(ethnicity_raw == "African American")
asian = pd.get_dummies(ethnicity_raw == "Asian")
caucasian = pd.get_dummies(ethnicity_raw == "Caucasian")

african_american.head()

,False,True
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [39]:
# select one dummy for african_american
is_african_american = african_american[[1]]
is_african_american.head()

,True
0,0
1,0
2,0
3,0
4,0


In [40]:
# rename column of is_african_american
is_african_american.rename(columns = {True : "IsAfricanAmerican"}, inplace = True)
is_african_american.head()

,IsAfricanAmerican
0,0
1,0
2,0
3,0
4,0


In [41]:
# select one dummy for asian and caucasian
is_asian = asian[[1]]
is_caucasian = caucasian[[1]]

# rename columns
is_asian.rename(columns = {True : "IsAsian"}, inplace = True)
is_caucasian.rename(columns = {True : "IsCaucasian"}, inplace = True)

In [42]:
# create dataframe with all variables
data = pd.concat([data_without_categorical_variables, is_asian, is_caucasian, is_african_american], axis = 1, join = 'inner')
data.head()

,Balance,Income,Limit,Rating,Cards,Age,Education,IsMarried,IsStudent,IsMale,IsAsian,IsCaucasian,IsAfricanAmerican
0,333,14.891,3606,283,2,34,11,1,0,1,0,1,0
1,903,106.025,6645,483,3,82,15,1,1,0,1,0,0
2,580,104.593,7075,514,4,71,11,0,0,1,1,0,0
3,964,148.924,9504,681,3,36,11,0,0,0,1,0,0
4,331,55.882,4897,357,2,68,16,1,0,1,0,1,0


In [43]:
# extract features

# target variable, integer
#balance = np.array(data["Balance"])

# integer
#limit = np.array(data["Limit"])
#rating = np.array(data["Rating"])
#cards = np.array(data["Cards"])
#age = np.array(data["Age"])
#education = np.array(data["Education"])

# float
#income = np.array(data["Income"])


# binary, converted to dummy dataframe
#gender_df = pd.get_dummies(data["Gender"]) # np.array(pd.get_dummies(data["Gender"]))
#student_df = pd.get_dummies(data["Student"]) # np.array(pd.get_dummies(data["Student"]))
#married_df = pd.get_dummies(data["Married"])

# categorical
#ethnicity_raw = pd.Series(data["Ethnicity"], dtype="category") 
#ethnicity_raw.head()


# TODO: weiter erkunden, visualisieren etc...?

### b) Datensatz teilen
Separieren Sie den Datensatz in 80%-20% für Training und Testdatensatz. Was müssen Sie hier beachten?

In [44]:
# split into dependent y and independent X variables
X = data.iloc[:, 1:]
y = data.iloc[:, 1]

In [48]:
# 80/20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Es muss beachtet werden, dass die Daten geshuffelt und dann separiert werden, sodass die Daten iid sind und beispielsweiße keine unterschiedliche Varianz oä in den zwei Splits vorhanden ist. 

### c) OLS mit allen Variablen und ohne Limit
Erstellen Sie aus der Variable Ethnicity drei binäre Variablen Ethnicity_Caucasian, Ethnicity_Asian und Ethnicity_African. Führen Sie eine lineare Regression mit OLS als Schätzer mit diesen Variablen und allen anderen Variablen (außer Limit) durch. Berechnen Sie Root Mean Square Error (RMSE) zwischen der Vorhersage und den wahren Werten im Test-Datensatz. Beschreiben Sie Ihre Ergebnisse.

Binäre Variablen wurde bereits in Teilaufgabe a) erstellt.

In [92]:
def ols(X_train, y_train, X_test, y_test):
    # Create linear regression object
    regr = linear_model.LinearRegression()

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    coef = regr.coef_
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return coef, mse, r2

#### OLS mit allen Variablen

In [93]:
# ols
coef_allVars, mse_allVars, r2_allVars = ols(X_train, y_train, X_test, y_test)

# The coefficients
print('Coefficients: \n', coef_allVars)
# The mean squared error
print("Mean squared error: %.2f"
      % mse_allVars)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_allVars)


Coefficients: 
 [ 1.00000000e+00 -4.47857851e-15 -3.09058479e-16 -1.09996553e-16
  2.88994130e-16 -2.00783316e-17  1.21881944e-15  7.66358615e-16
 -9.43618581e-16  3.05887372e-16 -6.99923108e-16  3.93860211e-16]
Mean squared error: 0.00
Variance score: 1.00


#### OLS ohne Limit

In [96]:
# drop column "Limit"
X_train_withoutLimit = X_train.drop(columns="Limit")
X_test_withoutLimit = X_test.drop(columns="Limit")

# ols
coef_withoutLimit, mse_withoutLimit, r2_withoutLimit = ols(X_train_withoutLimit, y_train, X_test_withoutLimit, y_test)

# The coefficients
print('Coefficients: \n', coef_withoutLimit)
# The mean squared error
print("Mean squared error: %.2f"
      % mse_withoutLimit)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_withoutLimit)

Coefficients: 
 [ 1.00000000e+00 -8.77129443e-16 -2.11665978e-16 -1.28447367e-16
 -1.82746605e-17  1.15951885e-15  7.30056211e-16 -9.66934788e-16
  3.40722461e-16 -7.14808478e-16  3.74685578e-16]
Mean squared error: 0.00
Variance score: 1.00


# TODO explain: perfect regression????

### d) OLS ohne Variable Ethnicity_Caucasian
Führen Sie die Analyse in 4c) erneut durch, ohne die Variable Ethnicity_Caucasian. Vergleichen Sie diese Ergebnisse mit denen aus 4c).

In [97]:
# drop column "IsCaucasian"
X_train_withoutIsCaucasian = X_train.drop(columns="IsCaucasian")
X_test_withoutIsCaucasian = X_test.drop(columns="IsCaucasian")

# ols
coef_withoutIsCaucasian, mse_withoutIsCaucasian, r2_withoutIsCaucasian = ols(X_train_withoutIsCaucasian, y_train, X_test_withoutIsCaucasian, y_test)

# The coefficients
print('Coefficients: \n', coef_withoutIsCaucasian)
# The mean squared error
print("Mean squared error: %.2f"
      % mse_withoutIsCaucasian)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_withoutIsCaucasian)

Coefficients: 
 [ 1.00000000e+00 -4.49123456e-15 -2.44697956e-16 -1.10039807e-16
  3.07296745e-16 -2.03119421e-17  1.21839225e-15  7.66504788e-16
 -9.43677664e-16  1.00475961e-15  1.09230094e-15]
Mean squared error: 0.00
Variance score: 1.00


# TODO: beschreiben und mit 4c vergleichen

### e) OLS mit unterschiedlichen Variablen
Führen Sie eine lineare Regression (OLS) mit folgenden Variablendurch: 
1. Income, Limit, Age
2. Income, Rating, Age
3. Income, Limit, Rating, Age 

Beschreiben Sie die Koeffizienten, Konfidenzintervalle und Signifikanzniveau. Vergleichen Sie die Ergebnisse.

#### 1. OLS mit Income, Limit, Age

In [98]:
# select features
cols_4e_1 = ["Income", "Limit", "Age"]
X_train_4e_1 = X_train[cols_4e_1]
X_test_4e_1 = X_test[cols_4e_1]

# ols
coef_4e_1, mse_4e_1, r2_4e_1 = ols(X_train_4e_1, y_train, X_test_4e_1, y_test)

# The coefficients
print('Coefficients: \n', coef_4e_1)
# The mean squared error
print("Mean squared error: %.2f"
      % mse_4e_1)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_4e_1)

Coefficients: 
 [ 1.00000000e+00 -2.89376180e-19 -7.25603903e-17]
Mean squared error: 0.00
Variance score: 1.00


#### 2. OLS mit Income, Rating, Age

In [99]:
# select features
cols_4e_2 = ["Income", "Rating", "Age"]
X_train_4e_2 = X_train[cols_4e_2]
X_test_4e_2 = X_test[cols_4e_2]

# ols
coef_4e_2, mse_4e_2, r2_4e_2 = ols(X_train_4e_2, y_train, X_test_4e_2, y_test)

# The coefficients
print('Coefficients: \n', coef_4e_2)
# The mean squared error
print("Mean squared error: %.2f"
      % mse_4e_2)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_4e_2)

Coefficients: 
 [ 1.00000000e+00 -5.79831281e-17 -5.38410862e-17]
Mean squared error: 0.00
Variance score: 1.00


#### 3. OLS mit Income, Limit, Rating, Age

In [100]:
# select features
cols_4e_3 = ["Income", "Limit", "Rating", "Age"]
X_train_4e_3 = X_train[cols_4e_3]
X_test_4e_3 = X_test[cols_4e_3]

# ols
coef_4e_3, mse_4e_3, r2_4e_3 = ols(X_train_4e_3, y_train, X_test_4e_3, y_test)

# The coefficients
print('Coefficients: \n', coef_4e_3)
# The mean squared error
print("Mean squared error: %.2f"
      % mse_4e_3)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_4e_3)

Coefficients: 
 [ 1.00000000e+00 -7.10617468e-16 -5.76344455e-17  6.34012840e-17]
Mean squared error: 0.00
Variance score: 1.00


# TODO: beschreiben, vergleichen, Signifikanz und Konfidenzintervall

### f) Korrelation
Was ist Korrelation? Wann benutzt man Pearson Korrelation, Spearman Korrelation und Kendalls Tau? Berechnen Sie Korrelationen zwischen allen Variablen im Trainings-Datensatz. Welche Schlussfolgerung können Sie daraus ziehen?

Anmerkung: $*$: Multiplikationszeichen
Auszüge aus Ausschitte aus dem Skript zur Vorlesung Statistik für Fortgeschrittene WS 2018/2019 Lehrstuhl für Analytics and Statistics (Prof. Grothe KIT), S. 58-61)
#### Was ist Korrelation?
Korrelation misst die Stärke und Richtung des Zusammenhangs bzw. der Abhängigkeit zwischen zwei Zufallsvariablen. 
Die Korrelation ist nicht mit Kausalität gleichzusetzen, da zwar ein Zusammenhang zwischen zwei Zufallsvariablen möglich ist, ohne dass diese sich direkt beeinflussen.

#### Pearson Korrelation
Das bekannteste skalare Zusammenhangsmaß zweier Zufallsvariablen X und Y ist der Korrelationskoeffizient
nach Bravais-Pearson:

$$Corr(X,Y) = \rho_{BP} = \dfrac{Cov(X,Y)}{\sqrt{X}*\sqrt{Y}}$$

Er misst die Stärke des linearen Zusammenhangs von X und Y und hängt von den
Randverteilungen von X und Y ab. 
Sein Wertebereich ist $[-1,1]$.

#### Spearman Korrelation
Das Korrelationsmaß Spearmans $\rho_S$ ist unabhängig von der Randverteilung und somit ein Rangkorrelationskoeffizient.
$$\rho_S = \dfrac{Cov(F_X(X), F_Y(Y))}{\sqrt{Var(F_X(X))}*\sqrt{F_Y(Y)}} = \dfrac{Cov(U, V)}{\sqrt{Var(U)}*\sqrt{Var(V)}}$$

$F_Z(Z)$ ist die Verteilungsfunktion der Zufallsvariable $Z$

Als Rangkorrelationskoeffizient ist Spearmans's $\rho_S$ invariant gegenüber streng monoton wachsende Transformationen $\alpha(.)$ und $\beta(.)$ der Ränder: 

$$ \rho_{S,X,Y} = \rho_{S,\alpha(x), \beta(Y)} $$

Auch hier ist der Wertebereich $[-1,1]$, jedoch wird nicht die Stärke des linearen Zusammenhangs zwischen den Zufallsvariablen X und Y, sondern zwischen U und V und somit den monotonen Zusammenhangs zwischen X und Y.


##### Kendalls $\tau$
Seien X, Y Zufallsvariablen mit gemeinsamer Verteilungsfunktion F X,Y und Copula C.
Mit $(X^∗ , Y^∗ )$ bezeichnen wir eine ”unabhängige Kopie” von $(X,Y)$.
Kendalls $\tau$ ist dann definiert durch:

$$\tau = P((X-X^*)((Y-Y^*))>0)-P((X-X^*)((Y-Y^*))<0)$$

Die erste Wahrscheinlichkeit heißt Konkordanz- die zweite Wahrscheinlichkeit Diskordanzwahrscheinlichkeit. Man kann zeigen, dass

$$ \tau = \dfrac{Cov(sign(X-X^*), sign(Y-Y^*))}{\sqrt{Var(sign(X-X^*))}\sqrt{Var(sign(Y-Y^*))}}$$

mit $$sign(z)= \left\{ \begin{array}{c} 1\quad z>0\\0 \quad   z=0\\-1 \quad   z<0 \end{array} \right.$$

Auch Kendalls $\tau$ ist ein Rangkorrelationskoeffizient.

#### Wann verwendet man was
Rangkorrelationsmaße können auch für ordinalskalierte Daten verwendet werden.
Ein linearer Zusammenhang impliziert einen monotonen Zusammenhang. Umgekehrt gilt dies nicht.
Ein Rangkorrelationsmaß ist zudem robuster gegenüber Ausreißer.

#### Berechnungen und Schlussfolgerung

In [104]:
# Pearson Correlation
corr_pearson = X.corr('pearson')
corr_pearson

,Income,Limit,Rating,Cards,Age,Education,IsMarried,IsStudent,IsMale,IsAsian,IsCaucasian,IsAfricanAmerican
Income,1.000000,0.792088,0.791378,-0.018273,0.175338,-0.027692,0.035652,0.019632,0.010738,-0.017137,-0.019701,0.040132
Limit,0.792088,1.000000,0.996880,0.010231,0.100888,-0.023549,0.031155,-0.006015,-0.009397,-0.032427,-0.003081,0.036320
Rating,0.791378,0.996880,1.000000,0.053239,0.103165,-0.030136,0.036751,-0.002028,-0.008885,-0.035999,-0.001070,0.037598
Cards,-0.018273,0.010231,0.053239,1.000000,0.042948,-0.051084,-0.009695,-0.026164,0.022658,0.005591,-0.005631,0.000878
Age,0.175338,0.100888,0.103165,0.042948,1.000000,0.003619,-0.073136,-0.029844,-0.004015,-0.059623,-0.000822,0.061169
Education,-0.027692,-0.023549,-0.030136,-0.051084,0.003619,1.000000,0.048911,0.072085,0.005049,0.029586,-0.037725,0.013827
IsMarried,0.035652,0.031155,0.036751,-0.009695,-0.073136,0.048911,1.000000,-0.076974,-0.012452,0.088595,0.011418,-0.102707
IsStudent,0.019632,-0.006015,-0.002028,-0.026164,-0.029844,0.072085,-0.076974,1.000000,-0.055034,0.053534,-0.048334,0.001931
IsMale,0.010738,-0.009397,-0.008885,0.022658,-0.004015,0.005049,-0.012452,-0.055034,1.000000,-0.025425,0.009831,0.014288
IsAsian,-0.017137,-0.032427,-0.035999,0.005591,-0.059623,0.029586,0.088595,0.053534,-0.025425,1.000000,-0.582131,-0.335526


In [105]:
# Kendalls Correlation
corr_kendall = X.corr('kendall')
corr_kendall

,Income,Limit,Rating,Cards,Age,Education,IsMarried,IsStudent,IsMale,IsAsian,IsCaucasian,IsAfricanAmerican
Income,1.000000,0.477360,0.473671,-0.040239,0.098550,-0.033921,0.010064,0.007994,-0.004109,-0.034315,0.015594,0.016590
Limit,0.477360,1.000000,0.947251,-0.001287,0.041427,-0.023127,0.027269,0.007818,-0.019714,-0.031596,0.018144,0.010890
Rating,0.473671,0.947251,1.000000,0.030216,0.044283,-0.029293,0.034112,0.008091,-0.018667,-0.033860,0.018532,0.012727
Cards,-0.040239,-0.001287,0.030216,1.000000,0.033210,-0.041765,-0.017770,-0.024230,0.009092,0.001054,-0.008607,0.008908
Age,0.098550,0.041427,0.044283,0.033210,1.000000,0.006612,-0.063600,-0.023315,-0.007035,-0.046894,-0.001909,0.049574
Education,-0.033921,-0.023127,-0.029293,-0.041765,0.006612,1.000000,0.037535,0.066235,0.002447,0.024038,-0.027643,0.007749
IsMarried,0.010064,0.027269,0.034112,-0.017770,-0.063600,0.037535,1.000000,-0.076974,-0.012452,0.088595,0.011418,-0.102707
IsStudent,0.007994,0.007818,0.008091,-0.024230,-0.023315,0.066235,-0.076974,1.000000,-0.055034,0.053534,-0.048334,0.001931
IsMale,-0.004109,-0.019714,-0.018667,0.009092,-0.007035,0.002447,-0.012452,-0.055034,1.000000,-0.025425,0.009831,0.014288
IsAsian,-0.034315,-0.031596,-0.033860,0.001054,-0.046894,0.024038,0.088595,0.053534,-0.025425,1.000000,-0.582131,-0.335526


In [106]:
# Spearman Correlation
corr_spearman = X.corr('spearman')
corr_spearman

,Income,Limit,Rating,Cards,Age,Education,IsMarried,IsStudent,IsMale,IsAsian,IsCaucasian,IsAfricanAmerican
Income,1.000000,0.657411,0.653274,-0.054929,0.147892,-0.046078,0.012310,0.009779,-0.005026,-0.041974,0.019075,0.020293
Limit,0.657411,1.000000,0.995760,-0.000837,0.064122,-0.032810,0.033353,0.009562,-0.024112,-0.038646,0.022192,0.013320
Rating,0.653274,0.995760,1.000000,0.043463,0.067763,-0.041426,0.041686,0.009887,-0.022812,-0.041378,0.022647,0.015552
Cards,-0.054929,-0.000837,0.043463,1.000000,0.046673,-0.052447,-0.019804,-0.027004,0.010133,0.001174,-0.009593,0.009928
Age,0.147892,0.064122,0.067763,0.046673,1.000000,0.011273,-0.077186,-0.028296,-0.008537,-0.056912,-0.002317,0.060164
Education,-0.046078,-0.032810,-0.041426,-0.052447,0.011273,1.000000,0.044063,0.077753,0.002873,0.028218,-0.032450,0.009097
IsMarried,0.012310,0.033353,0.041686,-0.019804,-0.077186,0.044063,1.000000,-0.076974,-0.012452,0.088595,0.011418,-0.102707
IsStudent,0.009779,0.009562,0.009887,-0.027004,-0.028296,0.077753,-0.076974,1.000000,-0.055034,0.053534,-0.048334,0.001931
IsMale,-0.005026,-0.024112,-0.022812,0.010133,-0.008537,0.002873,-0.012452,-0.055034,1.000000,-0.025425,0.009831,0.014288
IsAsian,-0.041974,-0.038646,-0.041378,0.001174,-0.056912,0.028218,0.088595,0.053534,-0.025425,1.000000,-0.582131,-0.335526


# TODO: Interpretation

### g) Random Forest
Führen Sie die Analyse in 4e) mit Random Forest anstatt der linearen Regression durch. Variieren Sie dabei den Parameter random_statemit den Werten {1, 33, 135, 123, 99, 22} und einigen selbst gewählten Werten. Berechnen Sie MSE des Test-Datensatzes. Vergleichen Sie Feature Importance. Welche Schlussfolgerung können Sie daraus ziehen?

### h) Multikollinearität
Was ist Multikollinearität? Was sind ihre möglichen Ursachen und Auswirkungen? Wie kann man sie erkennen? Welche Maßnahmen gibt es, wenn Variablen in den Daten kollinear sind?

Multikollinearität liegt vor, wenn zwei oder mehr erklärende Variablen hoch untereinander korreliert sind. Mit zunehmender Multikollinearität wird das Verfahren um die Regressionsparameter zu schätzen instabil und die Interpretation des Modells uneindeutig. Zwar ist der Schätzer im Falle hoher Multikollinearität immer noch BLU (best linear unbiased), jedoch sind die Koeffizienten sehr ungenau geschätzt und das Konfindeninterval zwar gültig, jedoch sehr breit. Dies ist darauf zurückzuführen, dass der Einfluss der einzelnen Variablen schlecht isoliert werden kann.

Perfekte Mulikollinearität (exakte lineare Beziehung zwischen Regressoren) führt dazu, dass die Durchführung der linearen Regressionsalanyse unmöglich wird.
Erkennbar wird Multikollinearität durch ein hohes Bestimmtheitsmaß (z.B. $R²$) zusammen mit niedrigen t-Werten (wenig Signifikanz) für die Regressionsparameter. Zudem kann man direkt an der paarweise Korrelation zwischen den Regressoren (größer 0,7 oder 0,8) Multikollinearität erkennen. Eine weitere Möglichkeit sind Hilfsregressionen, bei denen die erklärenden Variablen aufeinander regressiert werden. Je höher das $R²$ dieser Hilsregression, desto größer das Multikollinearitätsproblem.

Ursache perfekter Multikollinearität sind beispielsweise falsch verwendete Dummyvariablen, also z.B. eine Variable isStudent und eine isNotStudent. Ursachen für beinahe Mutlikollinearität sind gemeinsame Trends oder z

Maßnahmen: für Prognosen stellt Multikollinearität kein Problem dar. Soll jedoch der Einfluss einer erklärenden Variable auf die abhängige Variable geschätzt werden, liegt ein Problem vor. Möglichkeiten: 
- Zusätzliche externe Informationen: beispielsweise Zeitreihen- und Querschnittsdaten kombinieren
- Transformation der Variablen: beispielsweise Bildung erster Differenzen bei trendbehafteten Zeitreihen
- Variable weglassen (kann gefährlich werden: wenn beispielsweise Raucher ihr Handy in der Hosentasche aufbewahren, da eine andere Taschen der Kleidung mit der Zigarrettenschachtel belegt ist. Würde man nun die Variable istRaucher weglassen und bestimmen, ob und welchen Zusammenhang das Handy in der Hosentasche auf das Krebsrisiko hat, könnten schnell falsche Schlüsse gezogen werden)


(https://www.uibk.ac.at/econometrics/einf/kap08la.pdf)
